Stage 2:  Data preprocessing stage to produce structured data in csv format also stored in Cloud Object Storage. As columns in the csv file we suggest date, geographic location, url, and sentiment analysis.

In [ ]:
import lithops

In [ ]:
from transformers import pipeline
from custom_snscrape.twitter import Tweet

# https://github.com/lithops-cloud/lithops/blob/master/docs/data_processing.md

def map_preprocess(obj):
    
    #print('Bucket: {}'.format(obj.bucket))
    print('Key: {}'.format(obj.key))
    #print('Partition num: {}'.format(obj.part))

    data_body_str = obj.data_stream.read().decode('utf-8')    
    tweet = Tweet.from_json(data_body_str)

    # this model is the best one, I tested the others available and they're quite bad...
    #classifier = pipeline('sentiment-analysis', model="nlptown/bert-base-multilingual-uncased-sentiment")
    #results = classifier(tweet.content)
    
    return { 'id': tweet.id, 'user': tweet.user, 'content': tweet.content, 'lang': tweet.lang, 'coordinates': tweet.coordinates }

In [ ]:
import csv
from io import StringIO

def reduce_to_csv(results, storage):
    
    column_names = results[0].keys()
    csv_buffer = StringIO()

    # with StringIO as csv_buffer: remove .close if this is used

    dict_writer = csv.DictWriter(csv_buffer, column_names)
    dict_writer.writeheader()
    dict_writer.writerows(results)

    storage.put_object(bucket='colab',
                       key='test.csv',
                       body=csv_buffer.getvalue())
    
    csv_buffer.close()

    print(csv_buffer)

    return "reduced result to csv output and stored it"

In [ ]:
data_location = ['colab/#coronavirus/', 'colab/#sars/', 'colab/#covid/'] # The / at the end is important TODO: Avoid repetition? Reuse tags from stage 1 and prefix them with colab/

with lithops.FunctionExecutor(runtime='gilarasa/lithops-cloudbutton-challenge-py3.9:0.4', runtime_memory=2048) as fexec:

  fexec.map_reduce(map_preprocess, data_location, reduce_to_csv) # obj_chunk_number param is used to divide the input string in chunks, so not to be used here 
  print(fexec.get_result())

In [ ]:
#!lithops logs poll
